# **M3_AI2_Alternativa: Web Scrapping de películas**



- Peredo Mescco, Elizabeth Evelin

In [ ]:
!pip install cssselect
!pip install requests-html

In [58]:
import requests
#from bs4 import BeautifulSoup
import urllib.request
from lxml import html, cssselect
import pandas as pd
from requests_html import HTMLSession, AsyncHTMLSession

# A partir de la siguiente URL donde se lista las comedias (comedy) en español (spanish): https://www.imdb.com/search/title/?genres=comedy&languages=es contesta las siguientes preguntas:

## 1.     Mediante el inspector de código del navegador, obtén y anota el selector CSS de los siguientes datos de la primera película:

a.     Nombre

b.     Puntuación del Rating (estrellas) de IMDB.

c.      Número de votos (votes).

In [59]:
url = "https://www.imdb.com/search/title/?genres=comedy&languages=es"

nombre_selector="#main > div > div.lister.list.detail.sub-list > div > div:nth-child(1) > div.lister-item-content > h3 > a"
puntuacion_selector="#main > div > div.lister.list.detail.sub-list > div > div:nth-child(1) > div.lister-item-content > div > div.inline-block.ratings-imdb-rating > strong"
votos_selector="#main > div > div.lister.list.detail.sub-list > div > div:nth-child(1) > div.lister-item-content > p.sort-num_votes-visible > span:nth-child(2)"

## 2.     Identifica la parte común a los 3 selectores CSS del apartado 1 y la parte específica (última parte no común del selector CSS).

In [60]:
page_css_select="div.lister-item-content"

nombre_css_select="h3 > a"
puntuacion_css_select="div.inline-block.ratings-imdb-rating > strong"
votos_css_select="p.sort-num_votes-visible > span:nth-child(2)"

## 3.     Construye la estructura de árbol de nodos sobre la página de búsqueda proporcionada al inicio del ejercicio:  tree = html.document_fromstring(source.read())

In [61]:
source = urllib.request.urlopen(url)
tree = html.document_fromstring(source.read())
#print(tree.text_content())
print(html.tostring(tree, pretty_print=True).decode())

<html xmlns:og="http://ogp.me/ns#" xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         

        <meta charset="utf-8">




        <script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>

<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
  <script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
        <title>Comedy,
Spanish
(Sorted by Popularity Ascending) - IMDb</title>
  <script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>

        <link rel="canonical" href="https://www.imdb.com/search/title/?genres=comedy&amp;languages=es">
        <meta property="o

## 4.     Crea una lista de resultados la siguiente consulta CSS:  resultado = tree.cssselect("div.lister-item-content")
¿Cuántos elementos tiene tu lista? :  len(resultado)

In [62]:
resultado = tree.cssselect("div.lister-item-content")
n=len(resultado)
print("La lista tiene "+str(n)+" elementos")

La lista tiene 50 elementos


## 5. Construye un bucle que recorra la lista: resultado y que en cada elemento: resultado_i de la lista, realice 3 búsquedas: busquea_1, búsqueda_2 y búsqueda_3, una para cada selector CSS, utilizando sólo la parte específica del selector del apartado 2.

In [63]:
movies = tree.cssselect("div.lister-item")[:3]

for i, movie in enumerate(movies):
  title = movie.cssselect(nombre_css_select)[0].text_content()
  rating = movie.cssselect(puntuacion_css_select)[0].text_content()
  votes = movie.cssselect(votos_css_select)[0].text_content()
  print("Película", i+1,"\n",
          "Título:", title,"\n",
          "Puntuación:", rating,"\n",
          "Votos:", votes,"\n")

Película 1 
 Título: 超難搞先生 
 Puntuación: 7.5 
 Votos: 36,671 

Película 2 
 Título: 巴比倫 
 Puntuación: 7.3 
 Votos: 90,997 

Película 3 
 Título: 鞋貓劍客2 
 Puntuación: 7.9 
 Votos: 97,673 



## 6.     Para cada búsqueda realizada: busquea_1, búsqueda_2 y búsqueda_3, revisa si la nueva lista obtenida tiene longitud mayor que cero y en caso positivo, selecciona el primer elemento de la lista y haz un print de su contenido de texto: busquea_1[0].text_content()

In [64]:
movies = tree.xpath("//div[@class='lister-item-content']")[:3]

for i, movie in enumerate(movies):
  print("Película", i+1)
  title = movie.xpath(".//h3[@class='lister-item-header']/a")
  if len(title) > 0:
    print("Título:", title[0].text_content())

  rating = movie.xpath(".//div[@class='ratings-bar']/div/strong")
  if len(rating) > 0:
    print("Puntuación:", rating[0].get('data-value'))

  votes = movie.xpath(".//p[@class='sort-num_votes-visible']/span[2]")
  if len(votes) > 0:
    print("Votos:", votes[0].text_content())

  print("\n")


Película 1
Título: 超難搞先生
Puntuación: None
Votos: 36,671


Película 2
Título: 巴比倫
Puntuación: None
Votos: 90,997


Película 3
Título: 鞋貓劍客2
Puntuación: None
Votos: 97,673




## 7.     Guarda los resultados obtenidos de forma iterativa en un data frame que tenga como resultado 50 filas y las 3 columnas con los datos solicitados en el apartado 1.

In [65]:
movies = tree.xpath("//div[@class='lister-item-content']")
df=[]

for movie in movies:
    title = movie.xpath(".//h3[@class='lister-item-header']/a")
    c1 = title[0].text_content() if len(title) > 0 else ""

    rating = movie.xpath(".//div[@class='ratings-bar']/div[@class='inline-block ratings-imdb-rating']")
    c2 = rating[0].get('data-value') if len(rating) > 0 else 0

    votes = movie.xpath(".//p[@class='sort-num_votes-visible']/span[@name='nv']")
    c3 = votes[0].text_content() if len(votes) > 0 else 0

    df.append({
        "Nombre": c1,
        "Puntuación": c2,
        "Votos": c3
    })

df_movies = pd.DataFrame(df)
df_movies.head(10)

,Nombre,Puntuación,Votos
0,超難搞先生,7.5,"36,671"
1,巴比倫,7.3,"90,997"
2,鞋貓劍客2,7.9,"97,673"
3,超吉任務,7,"104,837"
4,五星饗魘,7.2,"264,084"
5,子彈列車,7.3,"316,503"
6,摩登家庭,8.5,"441,510"
7,六人行,8.9,"1,013,938"
8,Haunted Mansion,0,0
9,Bones,7.8,"164,068"
